In [11]:
import pandas as pd
from sqlalchemy import create_engine

# Consultas de Metadatos: PostgreSQL

In [20]:
usuario = "userPSQL"
contraseña = "passPSQL"
host = "localhost"  
puerto = "5432"
base_datos = "postgres"

engine = create_engine(f"postgresql+psycopg2://{usuario}:{contraseña}@{host}:{puerto}/{base_datos}")

Devolver toda la informmación de todas las asignaturas

In [38]:
query = """SELECT * FROM ASIGNATURAS"""
df = pd.read_sql(query, engine)
df.head()

,Unnamed: 0,id,nombre,numero_creditos,agno_academico,semestre,idioma
0,0,615000370,Gestion de Proyectos de Sistemas de Informacion,6,Cuarto curso,Séptimo semestre,Castellano
1,1,615000714,Bases de Datos,9,Segundo curso,Tercer semestre,Castellano
2,2,615000701,Analisis Matematico,6,Primer curso,Primer semestre,Castellano
3,3,615000721,Seguridad de la Informacion,3,Segundo curso,Cuarto semestre,Castellano
4,4,615000319,Programacion de Hw Reconfigurable,6,Tercero curso,Sexto semestre,Castellano


Devolver todos los profesores que imparten la asignatura Robótica

In [22]:
query = """
       SELECT P.* 
       FROM PROFESORES P 
       JOIN PROFESORESASIGNATURAS AP ON AP.PROFESOR_ID = P.id
       JOIN ASIGNATURAS A ON A.ID = AP.ASIGNATURA_ID
       WHERE A.NOMBRE = 'Robótica';
    """
df = pd.read_sql(query, engine)
df.head()

,Unnamed: 0,id,nombre,correo_electronico
0,57,57,Jose Eugenio Naranjo,joseeugenio.naranjo@upm.e\ns
1,58,58,Alfredo Valle Barrio,alfredo.valle@upm.es


Devolver los profesores que impartan más de 3 asignaturas

In [37]:
query = """
       SELECT P.NOMBRE, COUNT(A.ID) AS NUM_ASIGNATURAS
       FROM PROFESORES P 
       JOIN PROFESORESASIGNATURAS AP ON AP.PROFESOR_ID = P.ID
       JOIN ASIGNATURAS A ON A.ID = AP.ASIGNATURA_ID
       GROUP BY P.NOMBRE
       HAVING COUNT(A.ID) > 3;
    """
df = pd.read_sql(query, engine)
df.head()

,nombre,num_asignaturas
0,Edgar Talavera Muñoz,5
1,Victor Jose Martinez,6
2,Sergio Alejandro D'antonio,4
3,Joaquin Entrialgo Castaño,5
4,Julio Cesar Hernandez,4


Devolver todas el profesor que imparte el mayor numero de asignaturas de la titulación Grado en Sistemas de Informacion

In [36]:
query = """
        SELECT P.NOMBRE,COUNT(DISTINCT A.ID) AS NUM_ASIGNATURAS
        FROM PROFESORES P
        JOIN PROFESORESASIGNATURAS AP ON AP.PROFESOR_ID = P.ID
        JOIN ASIGNATURAS A ON A.ID = AP.ASIGNATURA_ID
        JOIN TITULACIONESASIGNATURAS TA ON TA.ASIGNATURA_ID = A.ID
        JOIN TITULACIONES T ON T.ID = TA.TITULACION_ID
        WHERE T.NOMBRE = 'Grado en Sistemas de Informacion'
        GROUP BY  P.NOMBRE
        ORDER BY NUM_ASIGNATURAS DESC
        LIMIT 1;
    """
df = pd.read_sql(query, engine)
df.head()

,nombre,num_asignaturas
0,Agustin Yague Panadero,4


# Consultas de contenido: ElasticSearch

In [107]:
from elasticsearch import Elasticsearch
from sqlalchemy import create_engine 
import re
from sentence_transformers import SentenceTransformer
import json

directory = "Guias Docentes"
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

es = Elasticsearch("http://elasticsearch:9200")
index = 'guias_docentes'

## Buscar asignaturas que mencionen un tema en su descripción

Consulta tipo match, útil para text

In [117]:
query_text = "matemática aplicada"

res = es.search(
    index=index,
    size=5,
    query={
        "match": {
            "descripcion_asignatura": query_text
        }
    }
)

for hit in res.body["hits"]["hits"]:
    print(hit["_source"]["nombre_asignatura"],"(", hit["_source"]["id_asignatura"], ")", "-", hit["_source"]["descripcion_asignatura"], "\n")

Matemática Discreta II ( 615001010 ) - La asignatura Matemática Discreta II complementa la asignatura Matemática Discreta I del primer semestre. En
ella se siguen estudiando algunas de las estructuras discretas importantes en Matemáticas y en Computación,
fundamentalmente Grafos. Se realiza un estudio básico de casi todos los conceptos de Teoría de Grafos,
incidiendo especialmente en los aspectos algorítmicos. También se estudian conceptos generales de Complejidad
de Algoritmos y técnicas de optimización combinatoria en problemas algorítmicos de grafos. 

Matemática Discreta I ( 615001003 ) - Matemática Discreta I es una asignatura en la que se estudian algunas de las estructuras discretas básicas
importantes en Matemáticas y en Computación. Se introduce el concepto de relación sobre un conjunto y se
estudian las diferentes tipos de relaciones y sus propiedades. Estas relaciones son la base para poder desarrollar
posteriormente otras estructuras como la aritmética entera y modular, álg

## Listar todas las asignaturas que mencionen un tema en su descripción

In [125]:
query_text = "seguridad informática"

res = es.search(
    index=index,
    size=16,
    query={
        "match": {"descripcion_asignatura": query_text}
    }
)

vistos = set()

print("Listado de asignaturas cuya descripción contiene", f"'{query_text}'")
for hit in res.body["hits"]["hits"]:
    src = hit["_source"]
    nombre = src["nombre_asignatura"]
    if nombre not in vistos:
        print("-", nombre)
        vistos.add(nombre)


Listado de asignaturas cuya descripción contiene 'seguridad informática'
- Fundamentos de Seguridad
- Seguridad de las Tecnologías de la Información
- Seguridad en Sistemas y Redes
- Seguridad de la Informacion
- Fundamentos Fisicos de la Informatica
- Auditoria y Control Ti


In [139]:
vistos = set()
query_text = "CC10"
# Consulta nested
res = es.search(
    index=index,
    size=100,
    query={
        "nested": {
            "path": "competencias",
            "query": {
                "term": {
                    "competencias.codigo": query_text
                }
            }
        }
    },
    _source=["nombre_asignatura"]
)

print(f"Asignaturas que contienen la competencia {query_text}:")
for hit in res.body["hits"]["hits"]:
    nombre = hit["_source"].get("nombre_asignatura", "")
    if nombre and nombre not in vistos:
        print("-", nombre)
        vistos.add(nombre)

Asignaturas que contienen la competencia CC10:
- Taller de Sistemas Operativos
- Sistemas Operativos


## Competencias más comunes en las asignaturas

In [56]:
res = es.search(
    index=index,
    size=0,
    aggs={
        "competencias_mas_comunes": {
            "nested": {"path": "competencias"},
            "aggs": {
                "por_codigo": {
                    "terms": {"field": "competencias.codigo"}
                }
            }
        }
    }
)

for bucket in res.body["aggregations"]["competencias_mas_comunes"]["por_codigo"]["buckets"]:
    print(bucket["key"], "-", bucket["doc_count"])

CT2 - 31
CG01 - 27
CT8 - 22
CB02 - 21
CB01 - 20
CB05 - 20
CE1 - 20
CG06 - 20
CT4 - 20
CT6 - 20


## Extraer nombre, temario y conocimientos previos de una asignatura buscando semánticamente por su campo descripción

In [105]:
def print_subtemas(subtemas, nivel=1, max_nivel=3):
    if nivel > max_nivel:
        return
    indent = "  " * nivel
    for sub in subtemas:
        numero = sub.get("numero", "")
        titulo = sub.get("titulo", "")
        print(f"{indent}{numero} - {titulo}")
        sub_subtemas = sub.get("subtemas", [])
        if sub_subtemas:
            print_subtemas(sub_subtemas, nivel + 1, max_nivel)

In [144]:
query_text = "robots"
query_vector = model.encode(query_text).tolist()

# Búsqueda semántica
res = es.search(
    index=index,
    size=5,
    knn={
        "field": "descripcion_vector",
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 100
    },
    _source=["id_asignatura", "nombre_asignatura", "conocimientos_previos", "temario", "descripcion_asignatura"]
)

# Mostrar resultados
for hit in res.body["hits"]["hits"]:
    src = hit["_source"]
    nombre = src.get("nombre_asignatura", "")
    id_asignatura = src.get("id_asignatura", "")
    print(f"{nombre} - {id_asignatura}, (score={hit['_score']:.4f})\n")
    print(f"Conocimientos previos: {src.get('conocimientos_previos', '')}\n")

    temario = src.get("temario", [])
    if temario:
        print("Temario:")
        print_subtemas(temario, nivel=1, max_nivel=3)
    print("\n" + "-"*50 + "\n")



Robotica - 615000334, (score=0.7708)

Conocimientos previos: - Algebra
- Inteligencia Artificial
- Fundamentos De Programacion
3.2. Otros conocimientos previos recomendados para cursar la asignatura
- Sistemas operativos GNU/Linux
- Lenguaje de programación Python

Temario:
  1 - Introducción a la robótica
    1.1 - Definición de robot y tipos
    1.2 - Desarrollo histórico
    1.3 - Aplicaciones de la robótica
  2 - Percepción y actuación sobre el entorno
    2.1 - Sensores para la navegación
    2.2 - Actuadores específicos para la robótica
    2.3 - Visión artificial
    2.4 - Detección de obstáculos
  3 - Control, planificación y optimización
    3.1 - Control clásico y control borroso
    3.2 - Arquitecturas de control
    3.3 - Políticas de decisión, optimización y aprendizaje por refuerzo
  4 - Robótica software
    4.1 - Arquitecturas de robots software
    4.2 - Programación de softbots
    4.3 - Robotic Process Automation (RPA)

-----------------------------------------------

## Búsqueda semántica por conocimientos previos

In [145]:
query_text = "matemáticas básicas y álgebra"
query_vector = model.encode(query_text).tolist()

res = es.search(
    index=index,
    size=5,
    knn={
        "field": "conocimientos_previos_vector",
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 100
    },
    _source=["id_asignatura", "conocimientos_previos"]
)

for hit in res.body["hits"]["hits"]:
    print(f"{hit['_source']['id_asignatura']} (score={hit['_score']:.4f})")
    print(hit["_source"].get("conocimientos_previos", ""), "\n")

615000220 (score=0.7223)
- Analisis Matematico
3.2. Otros conocimientos previos recomendados para cursar la asignatura
- Conocimientos básicos de física: cinemática, dinámica, trabajo y energía.
- Haber cursado las asignaturas de física y matemáticas del bachillerato científico/tecnológico
- Conocimientos básicos de matemáticas: trigonometría, álgebra vectorial, cálculo diferencial e integral. 

615001052 (score=0.7098)
- Probabilidad Y Estadistica
- Analisis Matematico
- Fundamentos De Programacion
- Algebra
3.2. Otros conocimientos previos recomendados para cursar la asignatura
- We strongly recommend a good mathematical background (especially in linear algebra) and familiarity with
computational complexity and cryptography. 

615000239 (score=0.7097)
- Analisis Matematico
- Logica Y Matematica Discreta
3.2. Otros conocimientos previos recomendados para cursar la asignatura
- Cálculo elemental de probabilidades
- Cálculo diferencial.
- Series geométricas
- Función Gamma.
- Combinator

In [147]:
query_text = "aprendizaje profundo y redes neuronales"
query_vector = model.encode(query_text, normalize_embeddings=True).tolist()

res = es.search(
    index=index,
    size=5,
    query={
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": """
                    double score = 0.5 * cosineSimilarity(params.query_vector, 'descripcion_vector') +
                                   0.3 * cosineSimilarity(params.query_vector, 'competencias_vector') +
                                   0.2 * cosineSimilarity(params.query_vector, 'conocimientos_previos_vector');
                    return (score + 1) * 50;  // desplazamos y escalamos
                """,
                "params": {"query_vector": query_vector}
            }
        }
    },
    _source=["nombre_asignatura", "descripcion_asignatura", "competencias.texto", "conocimientos_previos"]
)

for hit in res.body["hits"]["hits"]:
    print(f"{hit['_source']['nombre_asignatura']} (score={hit['_score']:.4f})")
    print("Descripción:", hit["_source"].get("descripcion_asignatura", "")[:600])
    print("Competencias:")
    for comp in hit["_source"].get("competencias", []):
        print("-", comp["texto"])
    print("Conocimientos previos:", hit["_source"].get("conocimientos_previos", ""), "\n")
    print("\n" + "-"*50 + "\n")


Aprendizaje Automático II (score=58.8683)
Descripción: El aprendizaje automático es uno de los campos de estudio en inteligencia artificial que ha obtenido grandes
logros, por ejemplo, en problemas de visión artificial, de reconocimiento del lenguaje natural o de control de robots
autónomos. Dentro del aprendizaje automático, se pueden distinguir diferentes tipos de métodos que son
aplicables a diversas clases de problemas. Esta asignatura se centra principalmente en métodos basados en
redes de neuronas artificiales y en métodos de aprendizaje de modelos probabilísticos.
Dentro del enfoque basado en redes de neuronas artificiales, se presentan mét
Competencias:
- CB01 - Que los estudiantes hayan demostrado poseer y comprender conocimientos en un área de estudio que parte de la base de la educación secundaria general, y se suele encontrar a un nivel que, si bien se apoya en libros de texto avanzados, incluye también algunos aspectos que implican conocimientos procedentes de la vanguardi